In [1]:
#importing required libraries
import warnings
warnings.filterwarnings("ignore")
#image precessing
import cv2
#file handling
import os
#obvious
import numpy as np
import pandas as pd
#library to know the progress of for loops
from tqdm import tqdm
# from matplotlib import pyplot as plt
from keras.utils import np_utils
from sklearn.utils import shuffle

Using TensorFlow backend.


In [2]:
#the path where the required files are
x = []
y = []
IMG_SIZE = 64
base_dir = 'Gestures'
for i in tqdm(os.listdir(base_dir)):
    path = os.path.join(base_dir,i)
    for j in os.listdir(path):
        img = cv2.imread(os.path.join(path,j),0)
#         print(img.shape)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        img = img/255
        if img is not None:
            x.append(img)
            y.append(i)

100%|██████████| 7/7 [00:02<00:00,  2.71it/s]


In [3]:
x = np.array(x)
y = np.array(y)

In [4]:
x.shape,y.shape

((8192, 64, 64), (8192,))

In [5]:
y = np_utils.to_categorical(y)
print(y.shape)

(8192, 7)


In [6]:
x,y = shuffle(x,y,random_state = 0)

In [7]:
x = x.reshape(x.shape[0],IMG_SIZE,IMG_SIZE,1)
x.shape,y.shape

((8192, 64, 64, 1), (8192, 7))

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

In [9]:
#multi classification model
nClasses = 7

In [10]:
#constructing the model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(IMG_SIZE,IMG_SIZE,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
 
model.add(Flatten())
model.add(Dense(nClasses, activation='softmax'))

W0621 16:52:13.952240 140247685961536 deprecation_wrapper.py:119] From /home/utkarsh/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0621 16:52:13.972165 140247685961536 deprecation_wrapper.py:119] From /home/utkarsh/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0621 16:52:13.976145 140247685961536 deprecation_wrapper.py:119] From /home/utkarsh/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0621 16:52:14.007838 140247685961536 deprecation_wrapper.py:119] From /home/utkarsh/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0621 16:52:14.

In [11]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

W0621 16:52:14.133467 140247685961536 deprecation_wrapper.py:119] From /home/utkarsh/.local/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0621 16:52:14.165839 140247685961536 deprecation_wrapper.py:119] From /home/utkarsh/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16384)             0         
__________

In [12]:
model.fit(x,y,epochs =5,batch_size = 100)

W0621 16:52:14.376157 140247685961536 deprecation.py:323] From /home/utkarsh/.local/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
8192/8192 [==============================] - 36s 4ms/step - loss: 0.2608 - acc: 0.9099
Epoch 2/5
8192/8192 [==============================] - 34s 4ms/step - loss: 0.0079 - acc: 0.9979
Epoch 3/5
8192/8192 [==============================] - 34s 4ms/step - loss: 0.0018 - acc: 0.9998
Epoch 4/5
8192/8192 [==============================] - 32s 4ms/step - loss: 5.2086e-04 - acc: 1.0000
Epoch 5/5
8192/8192 [==============================] - 30s 4ms/step - loss: 2.9590e-04 - acc: 1.0000


In [13]:
loss2, acc2 = model.evaluate(x,y)

8192/8192 [==============================] - 7s 891us/step


In [14]:
acc2

1.0

In [15]:
model.save("Gesture-model-saved.h5")